# Pytorch VGG16（Cat and Dog）パターン使用版


【お題】
- 以下のデータセットを用いて、「抽出パターン適用ガイドライン」に従って機械学習モデルを構築する。
 - 「Cat and Dog」データセット
   - 画像数：16010(学習：12808、検証：3202）
   - 画像サイズ：64×64
   - クラス数：2（cats、dogs）
  - データセットの入手先：
   - https://www.kaggle.com/tongpython/cat-and-dog
    - cat-and-dog.zip

【提出物】
- モデル学習時のepochごとのAccurncy、loss値の推移が分かるもの。
- モデル構築後、検証用画像に対してモデルを適用した際の、Accurncyが分かるもの。

In [0]:
!nvidia-smi

## ■パターン用ライブラリを使う準備

In [0]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

In [0]:
%%time
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/'

import os
if not os.path.exists('tmp'): # 12時間ルール対策用フォルダ
    os.makedirs('tmp')

# 【使用例】
#sys.stdout = open('/content/drive/My Drive/tmp/result.txt', 'a')
# …とても時間のかかる処理（学習処理など）…
#sys.stdout.close()
#sys.stdout = sys.__stdout__

In [0]:
# VisdomはColaboratoryでは動かない？（ローカルで構築したマシン向け？）
#%%time
#! npm install -g localtunnel
#!python3 -m pip install visdom
#!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
#!lt --port 8076 >> url.txt 2>&1 &

In [0]:
#%%time
#import time
#time.sleep(5)
#! cat url.txt

In [0]:
# lib の親フォルダまで移動（libまで指定するとエラー出る）
%cd "/content/drive/My Drive/Colab Notebooks/Pytorch/TopSE2019_SWDEV"

from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [0]:
# ライブラリを使ってモデルを定義する
%%time
net = network.InceptionV3(class_size=2)

## ■データセットを整備する

In [0]:
import shutil
import zipfile
import glob

In [0]:
gdrive_dir = "/content/drive/My Drive/Colab Notebooks/dataset/"
download_dir = "/root/download/"
data_dir = "/root/data/"
zip_file_name = "cat-and-dog.zip"

!rm -rf $data_dir

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

In [0]:
# Colab VMのストレージにDLする
%%time
s = gdrive_dir + zip_file_name
shutil.copy(s, download_dir)

In [0]:
# Colab VMのストレージに解凍する
%%time
f = os.path.join(download_dir, zip_file_name)
with zipfile.ZipFile(f) as zip:
  zip.extractall(download_dir)

#### 学習データ

In [0]:
# ラベル名を確認する
src_dir = '/root/download/training_set/training_set/'
dst_dir = '/root/data/cat_and_dog/train/'

!rm -rf $dst_dir
os.makedirs(dst_dir)

os.chdir(src_dir)
label_dirs = os.listdir(path='.')
print(label_dirs)

In [0]:
# 所定フォルダに振り分ける
for dir_name in label_dirs:
  numof_data = len( os.listdir(src_dir + dir_name)) # ラベルフォルダ内のファイル数
  files = glob.glob(src_dir + dir_name + "/*.jpg")  # ラベルフォルダ内のファイルリスト

  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    d = dst_dir + dir_name # 振り分け先フォルダ
    if not os.path.exists(d):
      os.makedirs(d)
    shutil.move(file, d)

In [0]:
# 意図通りに振り分けられたか確認する（今回は目視で確認）
print("Train files in " + dst_dir)
for dir_name in label_dirs:
  numof_data = len( os.listdir(dst_dir + dir_name)) # ラベルフォルダ内のファイル数
  print(" " + dir_name + ": " + str(numof_data))

#### 評価データ

In [0]:
# ラベル名を確認する
src_dir = '/root/download/test_set/test_set/'
dst_dir = '/root/data/cat_and_dog/test/'

!rm -rf $dst_dir
os.makedirs(dst_dir)

os.chdir(src_dir)
label_dirs = os.listdir(path='.')
print(label_dirs)

In [0]:
# 所定フォルダに振り分ける
for dir_name in label_dirs:
  numof_data = len( os.listdir(src_dir + dir_name)) # ラベルフォルダ内のファイル数
  files = glob.glob(src_dir + dir_name + "/*.jpg")  # ラベルフォルダ内のファイルリスト

  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    d = dst_dir + dir_name # 振り分け先フォルダ
    if not os.path.exists(d):
      os.makedirs(d)
    shutil.move(file, d)

In [0]:
# 意図通りに振り分けられたか確認する（今回は目視で確認）
print("Validation files in " + dst_dir)
for dir_name in label_dirs:
  numof_data = len( os.listdir(dst_dir + dir_name)) # ラベルフォルダ内のファイル数
  print(" " + dir_name + ": " + str(numof_data))

### データセット、データローダを作成

In [0]:
%%time
dst_dir = '/root/data/cat_and_dog/'

batch_size = 128
transform = dl.pattern_transform(resize=299, HorizontalFlip=True, VerticalFlip=True, Rotation=False, Perspective=True, Crop=False, Erasing=True)

train_dataset = torchvision.datasets.ImageFolder(root = dst_dir + 'train/', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root = dst_dir + 'test/', transform=dl.simple_transform(resize=299))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size, suffle=True)

## ■モデル設定

In [0]:
%%time
model = architecture.CNN_Architecture(net)

In [0]:
%%time
import sys
sys.stdout = open('/content/drive/My Drive/tmp/result_use_pattern.txt', 'w')
model.train(data_loader, epoch_count=50, is_inception=True)
sys.stdout.close()
sys.stdout = sys.__stdout__

In [0]:
%%time
model.predict(data_loader['val'])